# Первое соревнование на 0.90732. Теория - во второй части
Шевкунов Кирилл, 594 группа
________________________________

In [1]:
import re
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, ShuffleSplit

from sklearn.preprocessing import normalize
from sklearn.feature_selection import RFE
%matplotlib inline

# Import Data

In [2]:
folder = "~/workspace/datasets/ml-mipt-spring2018-hw5-v1/"
data = pd.read_csv(folder + 'train.csv')
data.drop('id', axis = 1, inplace = True)
X_train = np.asarray(data[data.columns[range(1, data.shape[1])]], dtype = np.double)
Y_train = np.asarray(data[['label']], dtype = np.double).ravel()
data.head()

,label,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,...,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216
0,1,7,0,3,0,2,3,0,6,0,...,3,4,2,2,0,13,0,11,1,3
1,1,0,11,0,0,10,1,0,0,4,...,0,2,0,0,2,8,1,13,0,4
2,0,9,0,3,0,1,3,0,4,0,...,48,11,2,0,0,4,0,2,0,0
3,0,0,9,3,2,25,0,4,0,0,...,1,14,1,0,0,0,3,0,17,1
4,0,0,0,0,0,2,5,0,0,0,...,3,12,0,3,0,4,0,24,4,0


In [3]:
data.groupby(['label']).count()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216
label,,,,,,,,,,,,,,,,,,,,,
0,327,327,327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,327,327
1,573,573,573,573,573,573,573,573,573,573,...,573,573,573,573,573,573,573,573,573,573


# Feature transformation

In [4]:
def transform_id(X, X_train):
    return X

In [5]:
def transform_normed(X, X_train):
    return (X - X_train.mean(axis=0)) / X_train.std(axis=0)

In [6]:
def transform_scaled(X, X_train):
    return X / X_train.max(axis=0)

In [36]:
from sklearn.decomposition import PCA

In [55]:
pca_100 = PCA(n_components=100, random_state=17)
pca_150 = PCA(n_components=150, random_state=17)
pca_200 = PCA(n_components=200, random_state=17)
            
pca_100.fit(X_train)
pca_150.fit(X_train)
pca_200.fit(X_train)

def transform_pca_100(X, X_train):
    return pca_100.transform(X)

def transform_pca_150(X, X_train):
    return pca_150.transform(X)

def transform_pca_200(X, X_train):
    return pca_200.transform(X)

# Fit Methods

In [56]:
transformations = [
    transform_pca_100,
    transform_pca_150,
    transform_pca_200,
]

In [57]:
estimators = [
    SGDClassifier(max_iter=50000, tol=0.1, penalty='l1', alpha=0.1),
    SGDClassifier(max_iter=50000, tol=0.1, penalty='l2', alpha=0.1),
    # SVC(kernel='rbf', C = 0.1, gamma=2),
    SVC(kernel='linear', C = 0.1, gamma=2),
    SVC(kernel='sigmoid', C = 0.1, gamma=2),
    SVC(kernel='poly', C = 0.1, gamma=2),
    
    # SVC(kernel='rbf', C = 1, gamma=2),
    SVC(kernel='linear', C = 1, gamma=2),
    SVC(kernel='sigmoid', C = 1, gamma=2),
    SVC(kernel='poly', C = 1, gamma=2),
    
    # SVC(kernel='rbf', C = 0.1, gamma="auto"),
    SVC(kernel='linear', C = 0.1, gamma="auto"),
    SVC(kernel='sigmoid', C = 0.1, gamma="auto"),
    SVC(kernel='poly', C = 0.1, gamma="auto"),
    
    #SVC(kernel='rbf', C = 1, gamma="auto"),
    SVC(kernel='linear', C = 1, gamma="auto"),
    SVC(kernel='sigmoid', C = 1, gamma="auto"),
    SVC(kernel='poly', C = 11, gamma="auto"),
    
    #SVC(kernel='precomputed', C = 0.1, gamma=2),
    #LinearSVC(penalty = 'l1', loss='hinge'), 
    LinearSVC(penalty = 'l1', loss='squared_hinge', dual=False),
    LinearSVC(penalty = 'l2', loss='hinge'),
    LinearSVC(penalty = 'l2', loss='squared_hinge', dual=False),# 'hinge' or 'squared_hinge', 
    LogisticRegression(penalty='l1', C=0.2),
    LogisticRegression(penalty='l1', C=1),
    LogisticRegression(penalty='l2', C=0.2),
    LogisticRegression(penalty='l2', C=1),
]

In [58]:
grid_result = []
cv_strategy = ShuffleSplit(n_splits=10, test_size=0.2, random_state=33)
for transform_i, transform in enumerate(transformations):
    X_train_transformed = transform(X_train, X_train)
    transform_name = str(transform).split(" ")[1]
    for estimator_i, estimator in enumerate(estimators):
        %time scoring = cross_val_score(estimator, X_train_transformed, Y_train, scoring='roc_auc', cv=cv_strategy)
        estimator_name_short = str(estimator)
        score_mean = scoring.mean()
        score_std = scoring.std()
        grid_result.append((transform_i, estimator_i, score_mean, score_std))
        print ('{}\n #data = {}, mean = {}, std = {}\n'.format(
            estimator_name_short, transform_name, round(score_mean, 4), round(score_std, 4)))

CPU times: user 874 ms, sys: 1.75 s, total: 2.62 s
Wall time: 334 ms
SGDClassifier(alpha=0.1, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=50000, n_iter=None,
       n_jobs=1, penalty='l1', power_t=0.5, random_state=None,
       shuffle=True, tol=0.1, verbose=0, warm_start=False)
 #data = transform_pca_100, mean = 0.85, std = 0.0328

CPU times: user 67.1 ms, sys: 183 ms, total: 250 ms
Wall time: 31.3 ms
SGDClassifier(alpha=0.1, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', max_iter=50000, n_iter=None,
       n_jobs=1, penalty='l2', power_t=0.5, random_state=None,
       shuffle=True, tol=0.1, verbose=0, warm_start=False)
 #data = transform_pca_100, mean = 0.8883, std = 0.0289

CPU times: user 4.39 s, sys: 454 ms, total: 4.84 s
Wall time: 4.24 s
SVC(C=0.1, cache_size=200, class_weig

CPU times: user 2.1 s, sys: 472 ms, total: 2.57 s
Wall time: 1.92 s
SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
 #data = transform_pca_150, mean = 0.9108, std = 0.015

CPU times: user 555 ms, sys: 0 ns, total: 555 ms
Wall time: 554 ms
SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
 #data = transform_pca_150, mean = 0.6907, std = 0.0289

CPU times: user 465 ms, sys: 0 ns, total: 465 ms
Wall time: 464 ms
SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
 #data = transfor

CPU times: user 615 ms, sys: 0 ns, total: 615 ms
Wall time: 614 ms
SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
 #data = transform_pca_200, mean = 0.9588, std = 0.0176

CPU times: user 1.89 s, sys: 0 ns, total: 1.89 s
Wall time: 1.89 s
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
 #data = transform_pca_200, mean = 0.9116, std = 0.0142

CPU times: user 631 ms, sys: 0 ns, total: 631 ms
Wall time: 629 ms
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='sigmoid',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
 #data = transform_pc

In [59]:
grid_result = np.array(grid_result)
best_5 = grid_result[:, 2].argsort()[-5:][::-1]
for i in best_5:
    print(grid_result[i])
    transformation = transformations[int(grid_result[i, 0])]
    transform_name = str(transform).split(" ")[1]
    estimator_name = str(estimators[int(grid_result[i, 1])])
    score_mean =  grid_result[i, 2]
    score_std =  grid_result[i, 3]
    print ('{}\n #data = {}, mean = {}, std = {}\n'.format(
        estimator_name, transform_name, round(score_mean, 4), round(score_std, 4)))

[2.         4.         0.95876586 0.01759795]
SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
 #data = transform_pca_200, mean = 0.9588, std = 0.0176

[2.         7.         0.95876586 0.01759795]
SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
 #data = transform_pca_200, mean = 0.9588, std = 0.0176

[ 2.         13.          0.95875239  0.01760427]
SVC(C=11, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
 #data = transform_pca_200, mean = 0.9588, std = 0.0176

[ 2.         10.      

# Get final classifier

In [60]:
best_i = int(grid_result[:, 2].argmax())
print(grid_result[best_i])
transformation = transformations[int(grid_result[best_i, 0])]
estimator = estimators[int(grid_result[best_i, 1])]
print("Best:", transformation, estimator)
X_train_transformed = transformation(X_train, X_train)
estimator.fit(X_train_transformed, Y_train)

[2.         4.         0.95876586 0.01759795]
Best: <function transform_pca_200 at 0x7fa277265158> SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Some parameters fitting

In [69]:
estimator = SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

params = {
    "C":np.linspace(0.01, 5, 100),
}
gscv = GridSearchCV(estimator, params, scoring='roc_auc', n_jobs=-1, verbose=100)
gscv.fit(X_train_transformed, Y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.06040404040404041 ...........................................
[CV] C=0.06040404040404041 ...........................................
[CV] C=0.11080808080808081 ...........................................
[CV] C=0.06040404040404041 ...........................................
[CV] C=0.11080808080808081 ...........................................
Memmaping (shape=(900, 200), dtype=float64) to new file /dev/shm/joblib_memmaping_pool_12182_140335761243440/12182-140336074714528-79edd677f0af76e3609fdc7b6552c595.pkl
Pickling array (shape=(900,), dtype=float64).
Pickling array (shape=(600,), dtype=int64).
Pickling array (shape=(300,), dtype=int64).
Memmaping (shape=(900, 200), dtype=float64) to old file /dev/shm/jo

[CV] .. C=0.16121212121212125, score=0.9675776934530957, total=   0.1s
[CV] .. C=0.21161616161616165, score=0.9675776934530957, total=   0.1s
[CV] C=0.3628282828282829 ............................................
[CV] C=0.3124242424242425 ............................................
[CV] .. C=0.16121212121212125, score=0.9080647485470004, total=   0.1s
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed:    0.4s
Memmaping (shape=(900, 200), dtype=float64) to old file /dev/shm/joblib_memmaping_pool_12182_140335761243440/12182-140336074714528-79edd677f0af76e3609fdc7b6552c595.pkl
Pickling array (shape=(900,), dtype=float64).
Pickling array (shape=(600,), dtype=int64).
Pickling array (shape=(300,), dtype=int64).
Pickling array (shape=(600,), dtype=int64).
Pickling array (shape=(300,), dtype=int64).
Memmaping (shape=(900, 200), dtype=float64) to old file /dev/shm/joblib_memmaping_pool_12182_140335761243440/12182-140336074714528-

[CV] .... C=0.614848484848485, score=0.9353955521398721, total=   0.1s
[CV] ... C=0.7156565656565658, score=0.9080647485470004, total=   0.1s
[CV] C=0.614848484848485 .............................................
[CV] ... C=0.7156565656565658, score=0.9353955521398721, total=   0.1s
[CV] C=0.7156565656565658 ............................................
[CV] C=0.7660606060606061 ............................................
[CV] ... C=0.6652525252525253, score=0.9675776934530957, total=   0.1s
[CV] C=0.6652525252525253 ............................................
[CV] ... C=0.7660606060606061, score=0.9675776934530957, total=   0.1s
[CV] C=0.7660606060606061 ............................................
[CV] .... C=0.614848484848485, score=0.9080647485470004, total=   0.1s
[CV] C=0.6652525252525253 ............................................
[CV] ... C=0.5644444444444445, score=0.9353955521398721, total=   0.1s
[CV] C=0.8164646464646466 ............................................
[CV] .

[Parallel(n_jobs=-1)]: Done  50 tasks      | elapsed:    1.3s
Memmaping (shape=(900, 200), dtype=float64) to old file /dev/shm/joblib_memmaping_pool_12182_140335761243440/12182-140336074714528-79edd677f0af76e3609fdc7b6552c595.pkl
Pickling array (shape=(900,), dtype=float64).
Pickling array (shape=(600,), dtype=int64).
Pickling array (shape=(300,), dtype=int64).
Pickling array (shape=(600,), dtype=int64).
Pickling array (shape=(300,), dtype=int64).
[Parallel(n_jobs=-1)]: Done  52 tasks      | elapsed:    1.4s
Memmaping (shape=(900, 200), dtype=float64) to old file /dev/shm/joblib_memmaping_pool_12182_140335761243440/12182-140336074714528-79edd677f0af76e3609fdc7b6552c595.pkl
Pickling array (shape=(900,), dtype=float64).
Pickling array (shape=(600,), dtype=int64).
Pickling array (shape=(300,), dtype=int64).
Pickling array (shape=(600,), dtype=int64).
Pickling array (shape=(300,), dtype=int64).
[Parallel(n_jobs=-1)]: Done  54 tasks      | elapsed:    1.4s
Memmaping (shape=(900, 200), dtype

[CV] ... C=1.3205050505050506, score=0.9675776934530957, total=   0.1s
[CV] ... C=1.4213131313131315, score=0.9080647485470004, total=   0.1s
[CV] ... C=1.4213131313131315, score=0.9353955521398721, total=   0.1s
[CV] C=1.5725252525252527 ............................................
[CV] C=1.4213131313131315 ............................................
[CV] C=1.471717171717172 .............................................
[CV] .... C=1.471717171717172, score=0.9675776934530957, total=   0.1s
[CV] C=1.471717171717172 .............................................
[CV] .... C=1.370909090909091, score=0.9353955521398721, total=   0.1s
[CV] C=1.622929292929293 .............................................
[CV] ... C=1.5221212121212122, score=0.9353955521398721, total=   0.1s
[CV] C=1.5221212121212122 ............................................
[CV] ... C=1.5221212121212122, score=0.9080647485470004, total=   0.1s
[CV] C=1.5725252525252527 ............................................
[CV] .

...

Memmaping (shape=(900, 200), dtype=float64) to old file /dev/shm/joblib_memmaping_pool_12182_140335761243440/12182-140336074714528-79edd677f0af76e3609fdc7b6552c595.pkl[CV] C=3.639090909090909 .............................................
[CV] ... C=3.4878787878787882, score=0.9675776934530957, total=   0.1s
[CV] C=3.4878787878787882 ............................................
[CV] ... C=3.5382828282828283, score=0.9353955521398721, total=   0.1s
[CV] ... C=3.5382828282828283, score=0.9080647485470004, total=   0.1s
[CV] C=3.5382828282828283 ............................................
[CV] C=3.5886868686868687 ............................................
[CV] ... C=3.4878787878787882, score=0.9353955521398721, total=   0.1s
[CV] C=3.6894949494949496 ............................................
[CV] ... C=3.5886868686868687, score=0.9675776934530957, total=   0.1s
[CV] C=3.5886868686868687 ............................................
[CV] .... C=3.437474747474748, score=0.9675776934530

[CV] ... C=3.8911111111111114, score=0.9353955521398721, total=   0.1s
[CV] C=4.092727272727273 .............................................
[CV] C=4.042323232323232 .............................................
[CV] .... C=3.941515151515152, score=0.9080647485470004, total=   0.1s
[CV] C=3.9919191919191923 ............................................
[CV] ... C=3.7903030303030305, score=0.9080647485470004, total=   0.1s
[CV] ... C=3.9919191919191923, score=0.9675776934530957, total=   0.1s
[CV] C=4.143131313131313 .............................................
[CV] C=3.9919191919191923 ............................................
[CV] ... C=3.8911111111111114, score=0.9080647485470004, total=   0.1s
[CV] .... C=3.941515151515152, score=0.9675776934530957, total=   0.1s
[CV] C=4.193535353535354 .............................................
[CV] C=4.143131313131313 .............................................
[CV] .... C=4.042323232323232, score=0.9353955521398721, total=   0.1s
Memmap

[CV] .... C=4.294343434343435, score=0.9675776934530957, total=   0.1s
[CV] C=4.445555555555556 .............................................
[CV] C=4.294343434343435 .............................................
[CV] .... C=4.344747474747475, score=0.9353955521398721, total=   0.1s
[CV] C=4.344747474747475 .............................................
[CV] .... C=4.344747474747475, score=0.9080647485470004, total=   0.1s
[CV] C=4.3951515151515155 ............................................
[CV] ... C=4.3951515151515155, score=0.9675776934530957, total=   0.1s
[CV] C=4.3951515151515155 ............................................
[CV] .... C=4.243939393939394, score=0.9675776934530957, total=   0.1s
[CV] C=4.495959595959596 .............................................
[CV] .... C=4.294343434343435, score=0.9353955521398721, total=   0.1s
[CV] C=4.546363636363637 .............................................
[CV] .... C=4.445555555555556, score=0.9353955521398721, total=   0.1s
[CV] .

[CV] .... C=4.798383838383839, score=0.9675776934530957, total=   0.1s
[CV] .... C=4.747979797979799, score=0.9675776934530957, total=   0.1s
[CV] C=5.0 ...........................................................
[CV] C=4.798383838383839 .............................................
[CV] ... C=4.8487878787878795, score=0.9080647485470004, total=   0.1s
[CV] C=4.899191919191919 .............................................
[CV] ... C=4.9495959595959595, score=0.9353955521398721, total=   0.1s
[CV] .... C=4.899191919191919, score=0.9675776934530957, total=   0.1s
[CV] C=4.9495959595959595 ............................................
[CV] C=4.899191919191919 .............................................
[CV] .... C=4.798383838383839, score=0.9353955521398721, total=   0.1s
[CV] ... C=4.9495959595959595, score=0.9080647485470004, total=   0.1s
[CV] C=5.0 ...........................................................
[CV] .................. C=5.0, score=0.9675776934530957, total=   0.1s
[CV] C

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'C': array([0.01  , 0.0604, ..., 4.9496, 5.    ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=100)

In [70]:
gscv.best_score_

0.9370126647133227

In [71]:
gscv.best_params_

{'C': 0.01}

In [72]:
gscv.best_estimator_

SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

# Finally get the best estimator

In [73]:
estimator = SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)  # << Зафиксировано для совместимости "наверняка"
estimator

SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [74]:
Y_train

array([1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0.,
       0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0.,
       0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0.,
       1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1.,
       0., 1., 1., 0., 0.

In [75]:
estimator.fit(X_train_transformed, Y_train)
estimator.predict(X_train_transformed)

array([1., 1., 0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.,
       0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0.,
       0., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0.,
       0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0.,
       1., 1., 0., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0.,
       0., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0.,
       1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1.,
       0., 1., 1., 0., 0.

# Test data estimation

In [76]:
test = pd.read_csv(folder + 'test.csv')
X_test = np.asarray(test[test.columns[range(1, test.shape[1])]])
X_test = transformation(X_test, X_train)
X_test

array([[ 4.90865372e+00,  8.07916915e+00, -3.03180384e+00, ...,
         3.21977423e-02, -9.24756792e-02, -2.10387762e-02],
       [ 3.05102364e+01, -1.22820387e+01,  9.36072067e+00, ...,
        -5.00214319e-02,  4.30124907e-02,  2.45761929e-02],
       [-1.03085217e+00,  7.40297910e+01,  1.62172706e+01, ...,
         3.92370449e-04,  7.13655978e-02, -1.21232583e-01],
       ...,
       [ 4.18816962e+01, -2.12295004e+01,  2.51458608e+01, ...,
         9.19711378e-02, -3.61865776e-02,  3.96493298e-02],
       [ 5.09818782e+01, -2.33857750e+01,  1.95681952e+01, ...,
        -1.85410226e-02, -2.44926707e-03, -1.18112231e-01],
       [ 2.55484472e+01, -4.53069025e+01, -2.65429521e+01, ...,
         3.13860331e-02,  9.26473448e-03, -1.50555213e-02]])

In [77]:
test_predict=estimator.predict(X_test)
test['label'] = test_predict
test[['id', 'label']].to_csv('sol_1.csv', sep = ',', index = False)
test

,id,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,...,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216,label
0,1,4,2,2,0,4,2,1,2,1,...,5,3,1,0,7,1,6,2,3,1.0
1,2,3,3,0,0,1,2,0,9,3,...,1,0,0,1,21,0,27,2,0,1.0
2,3,0,0,0,2,2,8,0,1,0,...,23,0,0,0,1,0,5,1,1,0.0
3,4,0,0,0,0,0,0,0,7,0,...,4,0,0,0,14,0,18,1,3,1.0
4,5,1,2,3,0,2,0,0,13,0,...,4,0,2,0,20,0,17,1,7,1.0
5,6,0,13,0,0,2,0,0,4,5,...,2,0,0,0,6,1,7,1,3,1.0
6,7,0,3,1,4,0,0,0,3,2,...,2,1,0,2,0,1,0,2,35,1.0
7,8,0,7,1,0,1,2,0,19,5,...,2,1,0,1,10,2,8,2,12,1.0
8,9,10,0,1,0,0,0,2,6,1,...,3,2,0,0,7,0,10,2,1,1.0
9,10,2,0,2,0,2,4,1,2,0,...,44,2,2,0,2,1,5,11,0,0.0


In [78]:
estimator, transformation

(SVC(C=0.01, cache_size=200, class_weight=None, coef0=0.0,
   decision_function_shape='ovr', degree=3, gamma=2, kernel='poly',
   max_iter=-1, probability=False, random_state=None, shrinking=True,
   tol=0.001, verbose=False), <function __main__.transform_pca_200>)